In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import sys

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from full_scanner import (
    get_sp500_tickers,
    get_nasdaq100_tickers,
    get_portfolio_tickers,
    scan_stocks,
    filter_buy_signals,
    create_excel_output,
    create_portfolio_excel,
    create_pdf_report
)

# Setup results directory
results_dir = ROOT / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("✓ Scanner modules loaded")
print(f"✓ Results directory: {results_dir}")
print(f"✓ Timestamp: {timestamp}")

✓ Scanner modules loaded
✓ Results directory: c:\workspace\portfolio_analyser\scanner_results
✓ Timestamp: 20260107_2127


## Configuration

Set your scan parameters:
- **daily_bars**: Number of daily bars to analyze (default: 60)
- **weekly_bars**: Number of weekly bars to analyze (default: 52)
- **concurrency**: Number of parallel workers (default: 2, max recommended: 5)

In [2]:
# Scan parameters
DAILY_BARS = 60
WEEKLY_BARS = 52
CONCURRENCY = 2  # Be careful with higher values (rate limits)

print(f"Daily bars: {DAILY_BARS}")
print(f"Weekly bars: {WEEKLY_BARS}")
print(f"Concurrency: {CONCURRENCY}")

Daily bars: 60
Weekly bars: 52
Concurrency: 2


## Option 1: Scan Portfolio Stocks Only

Quick scan of your portfolio stocks from `stocks.txt`. Includes ALL signals (not just FULL HOLD + ADD).

In [3]:
print("Loading portfolio tickers from stocks.txt...")
portfolio_tickers = get_portfolio_tickers()

if portfolio_tickers:
    print(f"Found {len(portfolio_tickers)} portfolio stocks\n")

    # Run scan
    portfolio_results = scan_stocks(
        portfolio_tickers,
        category="Portfolio",
        daily_bars=DAILY_BARS,
        weekly_bars=WEEKLY_BARS,
        concurrency=CONCURRENCY
    )

    if not portfolio_results.empty:
        print(f"\n✓ Scanned {len(portfolio_results)} stocks")

        # Count by signal
        signal_counts = portfolio_results['signal'].value_counts()
        print("\nSignal breakdown:")
        for signal, count in signal_counts.items():
            print(f"  {signal}: {count}")

        # Save results - use create_portfolio_excel for all signals
        xlsx_path = results_dir / f'portfolio_scanner_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_portfolio_{timestamp}.pdf'

        create_portfolio_excel(portfolio_results, xlsx_path, category="Portfolio")
        create_pdf_report(portfolio_results, portfolio_results, pdf_path, timestamp, category="Portfolio")

        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")

        # Show FULL HOLD + ADD stocks
        buy_signals = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD')
        if not buy_signals.empty:
            print(f"\n🎯 {len(buy_signals)} FULL HOLD + ADD signals:")
            # Display only columns that exist
            display_cols = ['ticker', 'signal', 'current_price']
            print(buy_signals[display_cols].to_string(index=False))
        else:
            print("\n⚠️ No FULL HOLD + ADD signals in portfolio")
else:
    print("⚠️ No portfolio tickers found in stocks.txt")

Loading portfolio tickers from stocks.txt...
[OK] Loaded 13 portfolio tickers from stocks.txt

Found 13 portfolio stocks

[SCAN] Scanning 13 Portfolio stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [1/13] TSLA   -> FULL HOLD + ADD      $431.84
[OK] [2/13] NVDA   -> FULL HOLD + ADD      $187.00
[OK] [6/13] PLTR   -> FULL HOLD + ADD      $179.57
[OK] [8/13] ASML   -> FULL HOLD + ADD      $1240.81
[OK] [12/13] MRVL   -> FULL HOLD + ADD      $87.81

[OK] Scan complete: 13 analyzed, 5 FULL HOLD + ADD signals found


✓ Scanned 13 stocks

Signal breakdown:
  FULL HOLD + ADD: 5
  HOLD MOST + REDUCE: 4
  CASH: 2
  HOLD: 2
âœ“ Excel file created: c:\workspace\portfolio_analyser\scanner_results\portfolio_scanner_20260107_2127.xlsx
  - All: 13 stocks
  - FULL HOLD + ADD: 5 stocks
  - HOLD MOST + REDUCE: 4 stocks
  - HOLD: 2 stocks
  - CASH: 2 stocks
[OK] PDF report created: c:\workspace\portfolio_analyser\scanner_results\scanner_report_portfolio_2

## Option 2: Scan S&P 500

Full scan of ~500 stocks. Takes 5-10 minutes depending on cache. Only outputs FULL HOLD + ADD signals.

In [4]:
print("Fetching S&P 500 ticker list...")
sp500_tickers = get_sp500_tickers()
print(f"Found {len(sp500_tickers)} S&P 500 stocks\n")

# Run scan
sp500_results = scan_stocks(
    sp500_tickers,
    category="S&P 500",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not sp500_results.empty:
    # Filter for buy signals only
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD')
    print(f"\n🎯 S&P 500: {len(sp500_buy)} FULL HOLD + ADD signals found")

    if not sp500_buy.empty:
        # Save results
        xlsx_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_sp500_{timestamp}.pdf'

        create_excel_output(sp500_buy, xlsx_path, category="S&P 500")
        create_pdf_report(sp500_buy, sp500_results, pdf_path, timestamp, category="S&P 500")

        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")

        # Display top results
        print("\nTop results:")
        display_cols = ['ticker', 'signal', 'current_price']
        print(sp500_buy[display_cols].head(10).to_string(index=False))
    else:
        print("\n⚠️ No FULL HOLD + ADD signals found in S&P 500")

Fetching S&P 500 ticker list...
[OK] Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

[SCAN] Scanning 503 S&P 500 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [4/503] ABBV   -> FULL HOLD + ADD      $223.93
[OK] [9/503] AFL    -> FULL HOLD + ADD      $111.96
[OK] [14/503] ALB    -> FULL HOLD + ADD      $158.15
[OK] [19/503] ALL    -> FULL HOLD + ADD      $207.92
[OK] [20/503] GOOGL  -> FULL HOLD + ADD      $314.34
[OK] [21/503] GOOG   -> FULL HOLD + ADD      $314.55
[OK] [23/503] AMZN   -> FULL HOLD + ADD      $240.93
[OK] [27/503] AXP    -> FULL HOLD + ADD      $383.56
[OK] [28/503] AIG    -> FULL HOLD + ADD      $78.06
[OK] [32/503] AME    -> FULL HOLD + ADD      $214.16
[OK] [33/503] AMGN   -> FULL HOLD + ADD      $330.17
[OK] [34/503] APH    -> FULL HOLD + ADD      $141.38
[OK] [35/503] ADI    -> FULL HOLD + ADD      $292.94
[OK] [38/503] APO    -> FULL HOLD + ADD      $152.70
[OK] [39/503] AAPL   -> FULL HOLD + ADD      $262

## Option 3: Scan NASDAQ 100

Scan ~100 tech-heavy stocks. Takes 2-5 minutes. Only outputs FULL HOLD + ADD signals.

In [5]:
print("Fetching NASDAQ 100 ticker list...")
nasdaq100_tickers = get_nasdaq100_tickers()
print(f"Found {len(nasdaq100_tickers)} NASDAQ 100 stocks\n")

# Run scan
nasdaq100_results = scan_stocks(
    nasdaq100_tickers,
    category="NASDAQ 100",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not nasdaq100_results.empty:
    # Filter for buy signals only
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD')
    print(f"\n🎯 NASDAQ 100: {len(nasdaq100_buy)} FULL HOLD + ADD signals found")

    if not nasdaq100_buy.empty:
        # Save results
        xlsx_path = results_dir / f'nasdaq100_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_nasdaq100_{timestamp}.pdf'

        create_excel_output(nasdaq100_buy, xlsx_path, category="NASDAQ 100")
        create_pdf_report(nasdaq100_buy, nasdaq100_results, pdf_path, timestamp, category="NASDAQ 100")

        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")

        # Display top results
        print("\nTop results:")
        display_cols = ['ticker', 'signal', 'current_price']
        print(nasdaq100_buy[display_cols].head(10).to_string(index=False))
    else:
        print("\n⚠️ No FULL HOLD + ADD signals found in NASDAQ 100")

Fetching NASDAQ 100 ticker list...
[OK] Loaded 101 NASDAQ 100 tickers

Found 101 NASDAQ 100 stocks

[SCAN] Scanning 101 NASDAQ 100 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [5/101] GOOGL  -> FULL HOLD + ADD      $314.34
[OK] [6/101] GOOG   -> FULL HOLD + ADD      $314.55
[OK] [8/101] AMGN   -> FULL HOLD + ADD      $330.17
[OK] [9/101] AMZN   -> FULL HOLD + ADD      $240.93
[OK] [10/101] ADI    -> FULL HOLD + ADD      $292.94
[OK] [11/101] AMAT   -> FULL HOLD + ADD      $296.01
[OK] [12/101] AAPL   -> FULL HOLD + ADD      $262.36
[OK] [13/101] APP    -> FULL HOLD + ADD      $617.24
[OK] [15/101] ASML   -> FULL HOLD + ADD      $1240.81
[OK] [16/101] AZN    -> FULL HOLD + ADD      $94.96
[OK] [22/101] BKNG   -> FULL HOLD + ADD      $5348.39
[OK] [27/101] CSCO   -> FULL HOLD + ADD      $75.23
[OK] [28/101] CCEP   -> FULL HOLD + ADD      $85.71
[OK] [29/101] CTSH   -> FULL HOLD + ADD      $84.62
[OK] [31/101] CEG    -> FULL HOLD + ADD 